# OpenCV Overlay: Filter2D and Dilate Example

This notebook illustrates the kinds of things you can do with accelerated openCV cores built as PYNQ overlay. The overlay consists of a 2D filter and a dilate function and this example notebook does the following.
1. Sets up HDMI drivers
2. Sets up widget for controlling different filter kernels
3. Run software only filter2D + dilate pipeline on HDMI input and output results on HDMI output
4. Run hardware accelerated dilate function
5. Run hardware accelerated filter2D + dilate function

NOTE: Rough FPS values are computed for each stage

Program overlay load python libraries for memory manager and accelerator drivers.

NOTE: All overlay and python libraries should be loaded prior to assigning the HDMI input/outputs. This is necessary right now to ensure correct functionality but will be enhanced in future releases. For now, please copy this block as is when using it in your own designs.

In [ ]:
from pynq.lib.video import *
from pynq.overlays.bare_hdmi import BareHDMIOverlay
base = BareHDMIOverlay("/home/xilinx/pynq/overlays/computer_vision/xv2Filter2DDilate.bit")
from pynq import Xlnk
mem_manager = Xlnk()
import pynq.overlays.xv2Filter2DDilate as xv2

hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

Setup and configure HDMI drivers (~10 seconds to initialize HDMI input/output)

In [ ]:
hdmi_in.configure(PIXEL_GRAY)
hdmi_out.configure(hdmi_in.mode)

hdmi_in.start()
hdmi_out.start()

Setup up HDMI input/output parameters which is referenced in later function calls

In [ ]:
mymode = hdmi_in.mode
print("My mode: "+str(mymode))

height = hdmi_in.mode.height
width = hdmi_in.mode.width
bpp = hdmi_in.mode.bits_per_pixel


Setup control widgets

Here, we define some kernel configurations that will be used to change the functionality of the 2D filter on the fly. A pulldown menu will appear below this cell to be used to change the filter2D kernel used subsequent cells.

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider, FloatSlider
import ipywidgets as widgets

kernelSize = 3

if kernelSize == 3:
    kernel_g = np.array([[0.0, 1.0, 0],[1.0, -4, 1.0],[0, 1.0, 0.0]],np.float32) #laplacian filter, high-pass  
else:
    kernel_g = np.array([[0,0,0,0,0],[0,0,1,0,0],[0,1,-4,1,0],[0,0,1,0,0],[0,0,0,0,0]],np.float32) #laplacian filter, high-pass  

def setKernelAndFilter3x3(kernelName):
    global kernel_g

    kernel_g = {
        'average blur':  np.ones((3,3),np.float32)/9.0,
        'gaussian blur': np.array([[0.0625,0.125,0.0625],[0.125,0.25,0.125],[0.0625,0.125,0.0625]]),
        'gaussian high-pass': np.array([[-0.0625,-0.125,-0.0625],[-0.125,0.75,-0.125],[-0.0625,-0.125,-0.0625]])*2,
        'laplacian high-pass':  np.array([[0.0, 1.0, 0],[1.0, -4, 1.0],[0, 1.0, 0.0]],np.float32),
        'Sobel Ver': np.array([[1.0,0.0,-1.0],[2.0,0.0,-2.0],[1.0,0.0,-1.0]],np.float32),
        'Sobel Hor': np.array([[1.0,2.0,1.0],[0.0,0.0,0.0],[-1.0,-2.0,-1.0]],np.float32)
    }.get(kernelName, np.ones((3,3),np.float32)/9.0)
    #print("kernel: "+str(kernelName))

def setKernelAndFilter5x5(kernelName):
    global kernel_g

    kernel_g = {
        'average blur':  np.ones((5,5),np.float32)/25.0,
        'gaussian blur': np.array([[0,0,0,0,0],[0,0.0625,0.125,0.0625,0],[0,0.125,0.25,0.125,0],[0,0.0625,0.125,0.0625,0],[0,0,0,0,0]]),
        'gaussian high-pass': np.array([[0,0,0,0,0],[0,-0.0625,-0.125,-0.0625,0],[0,-0.125,0.75,-0.125,0],[0,-0.0625,-0.125,-0.0625,0],[0,0,0,0,0]])*2,
        'laplacian high-pass':  np.array([[0,0,0,0,0],[0,0,1,0,0],[0,1,-4,1,0],[0,0,1,0,0],[0,0,0,0,0]],np.float32),
        'Sobel Ver': np.array([[0,0,0,0,0],[0,1,0,-1,0],[0,2,0,-2,0],[0,1,0,-1,0],[0,0,0,0,0]],np.float32),
        'Sobel Hor': np.array([[0,0,0,0,0],[0,1,2,1,0],[0,0,0,0,0],[0,-1,-2,-1,0],[0,0,0,0,0]],np.float32)
    }.get(kernelName, np.ones((5,5),np.float32)/25.0)
    #print("kernel: "+str(kernelName))

if kernelSize == 3: 
    interact(setKernelAndFilter3x3, kernelName=['average blur','gaussian blur','gaussian high-pass','laplacian high-pass','Sobel Hor','Sobel Ver']);
else:
    interact(setKernelAndFilter5x5, kernelName=['average blur','gaussian blur','gaussian high-pass','laplacian high-pass','Sobel Hor','Sobel Ver']);


Run SW Filter2D + Dilate (~30 seconds)

NOTE: In order to allow kernel redefintion onthe fly, subsequent function call are run as threads. This means you will not know if the cell is finished based on the cell status on the left. Be sure to wait until FPS information is reported before running other cells. Also note that if you use the widget to change the kernel, the FPS info will show up underneath the widget cell rather than the function block cell.

In [ ]:
import numpy as np
import cv2

def loop_sw2_app():
    global kernel_g

    kernelD = np.ones((3,3),np.uint8)
    buf =np.ones((height,width),np.uint8)

    numframes = 20

    start = time.time()
    for _ in range(numframes):
        inframe = hdmi_in.readframe()
        outframe = hdmi_out.newframe()
        cv2.filter2D(inframe, -1, kernel_g, dst=buf)
        cv2.dilate(buf, kernelD, dst=outframe, iterations=1)
        inframe.freebuffer()
        hdmi_out.writeframe(outframe)
    end = time.time()
    print("Frames per second:  " + str(numframes / (end - start)))
    
from threading import Thread

t = Thread(target=loop_sw2_app, )
t.start()

Run HW dilate only (~3 seconds)

Based on a kernel frequency of 142 MHz, this block should run at ~60 fps. This is also true for the filter2D block as well.

In [ ]:
import numpy as np

kernelVoid = np.zeros(0)

numframes = 200

start=time.time()
for _ in range(numframes):
    inframe = hdmi_in.readframe()
    outframe = hdmi_out.newframe()
    xv2.dilate(inframe, kernelVoid, dst=outframe, iterations=1, borderType=cv2.BORDER_CONSTANT)
    inframe.freebuffer()
    hdmi_out.writeframe(outframe)
end=time.time()
print("Frames per second:  " + str(numframes / (end - start)))

Run HW filter2D + dilate (~20 seconds)

Running both blocks in series means the effective performance is approximately halved or ~30 fps.

In [ ]:
import numpy as np

def loop_hw2_app():
    global kernel_g
    kernelVoid = np.zeros(0)
    
    buffer =  mem_manager.cma_alloc(length=width*height,data_type='unsigned char',cacheable=0)
    xFbuf = np.reshape(np.frombuffer(mem_manager.cma_get_buffer(buffer, length=width*height),dtype=np.uint8),(height,width))
    
    numframes = 600

    start=time.time()
    for _ in range(numframes):
        inframe = hdmi_in.readframe()
        outframe = hdmi_out.newframe()
        xv2.filter2D(inframe, -1, kernel_g, xFbuf, (-1,-1), 0.0, borderType=cv2.BORDER_CONSTANT)
        xv2.dilate(xFbuf, kernelVoid, dst=outframe, borderType=cv2.BORDER_CONSTANT)
        inframe.freebuffer()
        hdmi_out.writeframe(outframe)
    end=time.time()
    print("Frames per second:  " + str(numframes / (end - start)))

from threading import Thread

t = Thread(target=loop_hw2_app, )
t.start()

Clean up hdmi drivers

NOTE: This is needed to reset the HDMI drivers in a clean state. If this is not run, subsequent executions of this notebook may show visual artifacts on the HDMI out (usually a shifted output image)

In [ ]:
hdmi_out.close()
hdmi_in.close()